In [32]:
import pickle
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.utils import class_weight
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import GradientBoostingClassifier
import xgboost as xgb


In [2]:
df = pd.read_pickle('clean_crash_data.pkl')

In [3]:
df['SEVERITY'].value_counts()

SEVERITY
3    413351
2    195380
1      9961
4         4
Name: count, dtype: int64

In [4]:
df = df[df['SEVERITY'] != 4]
df['SEVERITY'].value_counts()

SEVERITY
3    413351
2    195380
1      9961
Name: count, dtype: int64

In [5]:
X = df[['SEX', 'AGE', 'HELMET_BELT_WORN', 'DAY_OF_WEEK', 'LIGHT_CONDITION', 'ROAD_GEOMETRY', 'SPEED_ZONE', 'SURFACE_COND', 'TOTAL_NO_OCCUPANTS', 'VEHICLE_YEARS_OLD']]
y = df['SEVERITY']
# Perform one-hot encoding for categorical variables
categorical_cols = ['SEX', 'HELMET_BELT_WORN', 'DAY_OF_WEEK', 'LIGHT_CONDITION', 'ROAD_GEOMETRY', 'SURFACE_COND']
encoder = OneHotEncoder(drop='first', sparse=False)
X_encoded = encoder.fit_transform(X[categorical_cols])
feature_names = encoder.get_feature_names_out(input_features=categorical_cols)
X_encoded_df = pd.DataFrame(X_encoded, columns=feature_names)
# Drop the original categorical columns and concatenate the encoded columns
X = pd.concat([X.drop(categorical_cols, axis=1).reset_index(drop=True), X_encoded_df.reset_index(drop=True)], axis=1)
# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)
# Create and train a logistic regression model



/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/preprocessing/_encoders.py:972: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


**Logistic Regression**

In [6]:
model = LogisticRegression()
model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = model.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy}')
print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))

Accuracy: 0.6688432911208269
              precision    recall  f1-score   support

           1       0.00      0.00      0.00      1992
           2       0.48      0.08      0.14     39076
           3       0.68      0.96      0.80     82671

    accuracy                           0.67    123739
   macro avg       0.39      0.35      0.31    123739
weighted avg       0.60      0.67      0.58    123739

[[    0   395  1597]
 [    0  3097 35979]
 [    0  3006 79665]]


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/metrics/_

In [7]:
feature_importance = abs(model.coef_[0])  # Absolute values for importance

# Map feature names to importance scores
feature_names = X_train.columns  # Replace with your actual feature names
feature_importance_dict = dict(zip(feature_names, feature_importance))

# Sort features by importance
sorted_features = sorted(feature_importance_dict.items(), key=lambda x: x[1], reverse=True)
sorted_features

[('TOTAL_NO_OCCUPANTS', 0.19980381069988604),
 ('ROAD_GEOMETRY_2', 0.18015537657172717),
 ('LIGHT_CONDITION_5', 0.15459870200908976),
 ('HELMET_BELT_WORN_9', 0.11223333168352376),
 ('DAY_OF_WEEK_4', 0.10836398181598901),
 ('ROAD_GEOMETRY_5', 0.09893691887044134),
 ('LIGHT_CONDITION_3', 0.09769794623456836),
 ('SURFACE_COND_2', 0.0803667586414338),
 ('DAY_OF_WEEK_3', 0.07849514765027113),
 ('DAY_OF_WEEK_5', 0.07694751931554067),
 ('HELMET_BELT_WORN_2', 0.05060355793871352),
 ('SURFACE_COND_9', 0.05016217487868456),
 ('LIGHT_CONDITION_2', 0.049207573359401735),
 ('DAY_OF_WEEK_2', 0.048780507169666046),
 ('DAY_OF_WEEK_6', 0.042126081962104336),
 ('ROAD_GEOMETRY_4', 0.024163478869716884),
 ('LIGHT_CONDITION_9', 0.020603168367946557),
 ('VEHICLE_YEARS_OLD', 0.01704147374157583),
 ('AGE', 0.014804630781804398),
 ('DAY_OF_WEEK_1', 0.010395645482561569),
 ('LIGHT_CONDITION_6', 0.008689386289348872),
 ('SEX_M', 0.008022491670732048),
 ('SPEED_ZONE', 0.007534469824564507),
 ('HELMET_BELT_WORN_8'

In [18]:

unique_actual, counts_actual = np.unique(y_test, return_counts=True)
unique_predicted, counts_predicted = np.unique(y_pred, return_counts=True)

# Create dictionaries to store the counts
actual_counts = dict(zip(unique_actual, counts_actual))
predicted_counts = dict(zip(unique_predicted, counts_predicted))

# Display the distribution of actual and predicted severities
print("Actual Severity Distribution:")
print(actual_counts)

print("\nPredicted Severity Distribution:")
print(predicted_counts)

Actual Severity Distribution:
{1: 1992, 2: 39076, 3: 82671}

Predicted Severity Distribution:
{1: 958, 2: 30514, 3: 92267}


Baseline Logistic Regression Seems to underpredict class 1 quite heavily, try improve our model through HyperParameter tuning and Class Weights

In [21]:
class_weights = dict(zip([1, 2, 3], class_weight.compute_class_weight('balanced', classes=[1, 2, 3], y=y_train)))

# Create and train a logistic regression model with class weights
weighted_model = LogisticRegression(class_weight=class_weights)
weighted_model.fit(X_train, y_train)
param_grid = {
    'C': [0.001, 0.01, 0.1, 1, 10, 100],  # Regularization parameter
    'penalty': ['l1', 'l2']  # Regularization type
}

# Create a GridSearchCV object
grid_search = GridSearchCV(LogisticRegression(class_weight=class_weights), param_grid, cv=5, scoring='f1_macro')

# Fit the GridSearchCV to your data
grid_search.fit(X_train, y_train)

# Get the best hyperparameters
best_params = grid_search.best_params_

# Train the model with the best hyperparameters
best_model = LogisticRegression(class_weight=class_weights, **best_params)
best_model.fit(X_train, y_train)

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://sciki

LogisticRegression(C=10,
                   class_weight={1: 20.703266825615927, 2: 1.055534940457911,
                                 3: 0.4989244385307044})

In [23]:
y_pred_best = best_model.predict(X_test)

# Evaluate the best model
accuracy_best = accuracy_score(y_test, y_pred_best)
print(f'Best Model Accuracy: {accuracy_best}')
print(classification_report(y_test, y_pred_best))
print(confusion_matrix(y_test, y_pred_best))

Best Model Accuracy: 0.4595236748317022
              precision    recall  f1-score   support

           1       0.04      0.61      0.08      1992
           2       0.35      0.30      0.32     39076
           3       0.74      0.53      0.62     82671

    accuracy                           0.46    123739
   macro avg       0.37      0.48      0.34    123739
weighted avg       0.60      0.46      0.52    123739

[[ 1225   402   365]
 [11979 11679 15418]
 [17124 21590 43957]]


In [24]:
feature_importance = abs(best_model.coef_[0])  # Absolute values for importance

# Map feature names to importance scores
feature_names = X_train.columns  # Replace with your actual feature names
feature_importance_dict = dict(zip(feature_names, feature_importance))

# Sort features by importance
sorted_features = sorted(feature_importance_dict.items(), key=lambda x: x[1], reverse=True)
sorted_features

[('LIGHT_CONDITION_5', 0.8117292650506047),
 ('HELMET_BELT_WORN_2', 0.4630536853820863),
 ('DAY_OF_WEEK_4', 0.46171440564044175),
 ('SEX_M', 0.41242014626485857),
 ('SURFACE_COND_2', 0.37261681645378114),
 ('SURFACE_COND_9', 0.36070848448397147),
 ('DAY_OF_WEEK_3', 0.3157415637655155),
 ('DAY_OF_WEEK_5', 0.3034232638405772),
 ('HELMET_BELT_WORN_6', 0.2664049804989535),
 ('LIGHT_CONDITION_2', 0.22569072997420708),
 ('DAY_OF_WEEK_2', 0.21428324726674763),
 ('ROAD_GEOMETRY_4', 0.18104784336669003),
 ('LIGHT_CONDITION_9', 0.14587979484783808),
 ('ROAD_GEOMETRY_5', 0.13593550283350003),
 ('DAY_OF_WEEK_6', 0.1230298959445845),
 ('ROAD_GEOMETRY_2', 0.11045343681218386),
 ('LIGHT_CONDITION_3', 0.10171099555022661),
 ('HELMET_BELT_WORN_9', 0.1013538422430687),
 ('DAY_OF_WEEK_1', 0.09429158006375235),
 ('TOTAL_NO_OCCUPANTS', 0.07567882380314442),
 ('HELMET_BELT_WORN_7', 0.0646683261832012),
 ('LIGHT_CONDITION_6', 0.057676174726637594),
 ('ROAD_GEOMETRY_3', 0.04453533035421775),
 ('HELMET_BELT_WO

**Random Forest**

In [25]:
from sklearn.ensemble import RandomForestClassifier

# Define class weights (adjust as needed)
class_weights = {1: 10, 2: 2, 3: 1}

# Create the classifier with class weights
clf = RandomForestClassifier(class_weight=class_weights)

clf.fit(X_train, y_train)

# Get feature importance scores
feature_importance = clf.feature_importances_

# Map feature names to importance scores
feature_names = X_train.columns  # Replace with your actual feature names
feature_importance_dict = dict(zip(feature_names, feature_importance))

# Sort features by importance
sorted_features = sorted(feature_importance_dict.items(), key=lambda x: x[1], reverse=True)


In [26]:
y_pred = clf.predict(X_test)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)

# Generate classification report
classification_rep = classification_report(y_test, y_pred)

# Generate confusion matrix
confusion_mat = confusion_matrix(y_test, y_pred)

# Print the results
print("Feature Importance:")
for feature, importance in sorted_features:
    print(f"{feature}: {importance:.4f}")

print("\nModel Performance on Test Data:")
print(f"Accuracy: {accuracy:.4f}")
print("Classification Report:\n", classification_rep)
print("Confusion Matrix:\n", confusion_mat)

Feature Importance:
AGE: 0.3882
VEHICLE_YEARS_OLD: 0.2735
SPEED_ZONE: 0.0843
TOTAL_NO_OCCUPANTS: 0.0457
SURFACE_COND_2: 0.0205
HELMET_BELT_WORN_9: 0.0144
LIGHT_CONDITION_2: 0.0141
ROAD_GEOMETRY_2: 0.0136
ROAD_GEOMETRY_5: 0.0133
SEX_M: 0.0111
DAY_OF_WEEK_5: 0.0101
DAY_OF_WEEK_6: 0.0100
DAY_OF_WEEK_2: 0.0094
DAY_OF_WEEK_3: 0.0092
LIGHT_CONDITION_3: 0.0090
DAY_OF_WEEK_4: 0.0090
LIGHT_CONDITION_5: 0.0080
DAY_OF_WEEK_1: 0.0078
DAY_OF_WEEK_7: 0.0075
SURFACE_COND_9: 0.0068
HELMET_BELT_WORN_6: 0.0060
ROAD_GEOMETRY_4: 0.0054
HELMET_BELT_WORN_2: 0.0053
LIGHT_CONDITION_6: 0.0023
LIGHT_CONDITION_9: 0.0022
HELMET_BELT_WORN_8: 0.0022
HELMET_BELT_WORN_5: 0.0016
LIGHT_CONDITION_4: 0.0015
HELMET_BELT_WORN_7: 0.0015
SURFACE_COND_5: 0.0015
SURFACE_COND_3: 0.0013
ROAD_GEOMETRY_3: 0.0012
SEX_U: 0.0010
ROAD_GEOMETRY_9: 0.0009
SURFACE_COND_4: 0.0003
ROAD_GEOMETRY_6: 0.0002
ROAD_GEOMETRY_7: 0.0000
ROAD_GEOMETRY_8: 0.0000
HELMET_BELT_WORN_4: 0.0000

Model Performance on Test Data:
Accuracy: 0.6747
Classificati

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

# Define the hyperparameter grid for the Random Forest
param_grid = {
    'n_estimators': [100, 200],
    'max_depth': [10, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2]
}

# Create a Random Forest classifier
clf = RandomForestClassifier(class_weight='balanced', random_state=42)

# Create a GridSearchCV object
grid_search = GridSearchCV(clf, param_grid, cv=5, scoring='f1_macro', n_jobs=-1)

# Fit the GridSearchCV to your data
grid_search.fit(X_train, y_train)

# Get the best hyperparameters
best_params = grid_search.best_params_

# Train the model with the best hyperparameters
best_model = RandomForestClassifier(class_weight='balanced', random_state=42, **best_params)
best_model.fit(X_train, y_train)

# Make predictions on the test set
y_pred_best = best_model.predict(X_test)

# Evaluate the best model
accuracy_best = accuracy_score(y_test, y_pred_best)
print(f'Best Model Accuracy: {accuracy_best}')
print(classification_report(y_test, y_pred_best))
print(confusion_matrix(y_test, y_pred_best))


In [ ]:


# Map class labels to start from 0
y_train_mapped = y_train - 1  # Subtract 1 from each class label to map to 0, 1, 2
y_test_mapped = y_test - 1

# Calculate class weights for balanced classes
class_weights = len(y_train_mapped) / (len(np.unique(y_train_mapped)) * np.bincount(y_train_mapped))

# Create a custom weight array for each sample in the training data
sample_weights = np.array([class_weights[label] for label in y_train_mapped])

# Define the XGBoost classifier
clf = xgb.XGBClassifier(random_state=42)

# Define the hyperparameter grid for the XGBoost model
param_grid = {
    'n_estimators': [100, 200],
    'learning_rate': [0.01, 0.1],
    'max_depth': [3, 4]
}

# Create a GridSearchCV object
grid_search = GridSearchCV(clf, param_grid, cv=5, scoring='f1_macro', n_jobs=-1)

# Fit the GridSearchCV to your data, passing the custom sample weights
grid_search.fit(X_train, y_train_mapped, sample_weight=sample_weights)

# Get the best hyperparameters
best_params = grid_search.best_params_

# Train the model with the best hyperparameters
best_model = xgb.XGBClassifier(random_state=42, **best_params)

# Fit the model using the custom sample weights
best_model.fit(X_train, y_train_mapped, sample_weight=sample_weights)

# Make predictions on the test set
y_pred_best = best_model.predict(X_test)

# Map predicted class labels back to 1, 2, 3
y_pred_best_mapped = y_pred_best + 1

# Evaluate the best model
accuracy_best = accuracy_score(y_test, y_pred_best_mapped)
print(f'Best Model Accuracy: {accuracy_best}')
print(classification_report(y_test, y_pred_best_mapped))
print(confusion_matrix(y_test, y_pred_best_mapped))
